# Ataca la API de Marvel

![imagen](https://cdn.hobbyconsolas.com/sites/navi.axelspringer.es/public/styles/hc_1440x810/public/media/image/2021/09/marvel-2460339.jpg?itok=cBnC1CBi)

Te tendrás que [registrar](https://developer.marvel.com/) y consultar la [documentación](https://developer.marvel.com/docs)

Queremos que consultes a la api para que te devuelva la info de los personajes de marvel que empiecen por la inicial de tu nombre. Tendríamos que generar un main.py (más los scripts que se te ocurran) que guarde un csv la información con la siguiente estructura

![imagen](./img/Captura_marvel.PNG)

Aquí te dejamos el código en python para poder empezar a hacer las llamadas, rellenando algunas variables como tus keys, parámetros de la llamada y la url (endpoint) a la que quieres acceder

In [1]:
import hashlib
import requests
import datetime
import pandas as pd

pd.set_option("max_colwidth", 100)
# pd.set_option("max_rows", 1000)

In [1]:
%pip install requests

   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ------------------- -------------------- 30.7/62.6 kB 660.6 kB/s eta 0:00:01
   -------------------------- ------------- 41.0/62.6 kB 495.5 kB/s eta 0:00:01
   -------------------------- ------------- 41.0/62.6 kB 495.5 kB/s eta 0:00:01
   -------------------------- ------------- 41.0/62.6 kB 495.5 kB/s eta 0:00:01
   ---------------------------------------  61.4/62.6 kB 234.9 kB/s eta 0:00:01
   ---------------------------------------- 62.6/62.6 kB 223.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/162.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/162.5 kB ? eta -:--:--
   ------- -------------------------------- 30.7/162.5 kB 1.4 MB/s eta 0:00:01
   --------- ----------------------------- 41.0/162.5 kB 667.8 kB/s eta 0:00:01
   ------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


,id,name,picture_url
0,1010914,Captain America (House of M),http://i.annihil.us/u/prod/marvel/i/mg/6/10/53176a1e7c0d5.jpg
1,1017295,Captain America (LEGO Marvel Super Heroes),http://i.annihil.us/u/prod/marvel/i/mg/d/b0/5239c38051946.jpg
2,1017327,Captain America (Marvel War of Heroes),http://i.annihil.us/u/prod/marvel/i/mg/2/03/5239c005a4827.jpg
3,1017575,Captain America (Sam Wilson),http://i.annihil.us/u/prod/marvel/i/mg/c/80/545a84a75ddaf.jpg
4,1010913,Captain America (Ultimate),http://i.annihil.us/u/prod/marvel/i/mg/5/80/53176a0bb810c.jpg
...,...,...,...
95,1011333,Dakota North,http://i.annihil.us/u/prod/marvel/i/mg/7/d0/4ce5a6888f769.jpg
96,1009259,Damage Control,http://i.annihil.us/u/prod/marvel/i/mg/5/00/4c00404b5e1a2.jpg
97,1009453,Dani Moonstar,http://i.annihil.us/u/prod/marvel/i/mg/6/20/4ce5a0614487a.jpg
98,1010776,Danny Rand,http://i.annihil.us/u/prod/marvel/i/mg/2/20/4c00374b1008a.jpg


,id,name,picture_url
0,1011334.0,3-D Man,http://i.annihil.us/u/prod/marvel/i/mg/c/e0/535fecbbb9784.jpg
1,1017100.0,A-Bomb (HAS),http://i.annihil.us/u/prod/marvel/i/mg/3/20/5232158de5b16.jpg
2,1009144.0,A.I.M.,http://i.annihil.us/u/prod/marvel/i/mg/6/20/52602f21f29ec.jpg
3,1010699.0,Aaron Stack,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
4,1009146.0,Abomination (Emil Blonsky),http://i.annihil.us/u/prod/marvel/i/mg/9/50/4ce18691cbf04.jpg
...,...,...,...
1295,1009631.0,Sue Storm,http://i.annihil.us/u/prod/marvel/i/mg/6/a0/52695b9cd40b6.jpg
1296,1011212.0,Sugar Man,http://i.annihil.us/u/prod/marvel/i/mg/f/d0/527413a2480b5.jpg
1297,1009635.0,Sumo,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
1298,1009636.0,Sunfire,http://i.annihil.us/u/prod/marvel/i/mg/7/00/53629b9e24ef5.jpg


,id,name,picture_url
0,1009637.0,Sunset Bain,http://i.annihil.us/u/prod/marvel/i/mg/c/10/4c003ab9ed7d6.jpg
1,1009638.0,Sunspot,http://i.annihil.us/u/prod/marvel/i/mg/b/50/4c003ab716add.jpg
2,1015280.0,Super Hero Squad,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
3,1011022.0,Super-Adaptoid,http://i.annihil.us/u/prod/marvel/i/mg/5/e0/4c00325af279c.jpg
4,1009639.0,Super-Skrull,http://i.annihil.us/u/prod/marvel/i/mg/9/b0/5274138fe6deb.jpg
...,...,...,...
257,1011515.0,Zeus,http://i.annihil.us/u/prod/marvel/i/mg/f/60/4ce5a7fcaa386.png
258,1011127.0,Zodiak,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
259,1009741.0,Zombie (Simon Garth),http://i.annihil.us/u/prod/marvel/i/mg/6/10/4c003937c9ba4.jpg
260,1011183.0,Zuras,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg


,id,name,picture_url
0,1011334.0,3-D Man,http://i.annihil.us/u/prod/marvel/i/mg/c/e0/535fecbbb9784.jpg
1,1017100.0,A-Bomb (HAS),http://i.annihil.us/u/prod/marvel/i/mg/3/20/5232158de5b16.jpg
2,1009144.0,A.I.M.,http://i.annihil.us/u/prod/marvel/i/mg/6/20/52602f21f29ec.jpg
3,1010699.0,Aaron Stack,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
4,1009146.0,Abomination (Emil Blonsky),http://i.annihil.us/u/prod/marvel/i/mg/9/50/4ce18691cbf04.jpg
...,...,...,...
1557,1011515.0,Zeus,http://i.annihil.us/u/prod/marvel/i/mg/f/60/4ce5a7fcaa386.png
1558,1011127.0,Zodiak,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
1559,1009741.0,Zombie (Simon Garth),http://i.annihil.us/u/prod/marvel/i/mg/6/10/4c003937c9ba4.jpg
1560,1011183.0,Zuras,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg


## Encapsúlalo en un main.py, funciones.py y variables.py